# ETL

In [48]:
import pandas as pd
import ast

Cargamos los datos

In [49]:
df_movies = pd.read_csv('../Datasets/movies_dataset.csv')
df_credits = pd.read_csv('../Datasets/credits.csv')

C:\Users\jaime\AppData\Local\Temp\ipykernel_8536\1939930400.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv('../Datasets/movies_dataset.csv')


Creamos las funciones para desanidar

In [50]:
def extraer_nombre_dict(columna):
    try:
        columna = ast.literal_eval(columna)
        if isinstance(columna, dict):
            return columna.get('name')
    except:
        return 'NA'
    else:
        return 'NA'

def extraer_nombre_list(columna):
    try:
        columna = ast.literal_eval(columna)
        if isinstance(columna, list):
            return [item.get('name') for item in columna if isinstance(item, dict)]
    except:
        return 'NA'
    else:
        return 'NA'

def extraer_directores(crew_list):
    try:
        crew_list = ast.literal_eval(crew_list)
        return [crew['name'] for crew in crew_list if crew.get('job') == 'Director']
    except:
        return 'NA'

Eliminamos columnas no utilizadas

In [51]:
df_movies = df_movies.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'])
df_credits = df_credits.drop(columns=['id'])

Unimos los dataframes

In [52]:
df = df_movies.join(df_credits)

Eliminamos registros duplicados

In [53]:
df = df.drop_duplicates()

Desanidamos columnas

In [54]:
df['belongs_to_collection'] = df['belongs_to_collection'].apply(extraer_nombre_dict)
df['genres'] = df['genres'].apply(extraer_nombre_list)
df['production_companies'] = df['production_companies'].apply(extraer_nombre_list)
df['production_countries'] = df['production_countries'].apply(extraer_nombre_list)
df['spoken_languages'] = df['spoken_languages'].apply(extraer_nombre_list)
df['cast'] = df['cast'].apply(extraer_nombre_list)
df['director'] = df['crew'].apply(extraer_directores)

Eliminamos datos no utilizados

In [55]:
df = df.drop(columns=['crew'])

Rellenamos valores nulos

In [56]:
df['revenue'] = df['revenue'].fillna(0)
df['budget'] = df['budget'].fillna(0)

Transformamos columnas

In [57]:
df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d', errors='coerce')
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')

Eliminamos registros con fechas nulas

In [58]:
df = df.dropna(subset=['release_date'])

Generamos columnas nuevas

In [59]:
df['release_year'] = df['release_date'].dt.year
df['return'] = df.apply(lambda row: row['revenue'] / row['budget'] if row['budget'] != 0 else 0, axis=1)

Recortamos el tamaño del dataframe

In [60]:
df = df.head(20000)

Guardamos los cambios

In [61]:
df.to_csv('../Datasets/movies_dataset_ETL.csv', index=False)